In [1]:
import pandas as pd

In [2]:
fh = '../files/tickets-gen-all.csv'

In [3]:
df = pd.read_csv(fh, index_col=0, parse_dates=['created', 'opened_at', 'updated_on'])

In [4]:
df.shape

(75724, 22)

In [5]:
df.columns

Index(['created', 'caller', 'opened_at', 'opened_by', 'priority', 'state',
       'category', 'subcategory', 'assignment_group', 'bpl_location',
       'nyp_locaiton', 'resolved', 'assigned_to', 'system', 'closed_at',
       'closed_by', 'mat_source', 'reassignment_count', 'reopen_count',
       'resolved_by', 'updated_on', 'updated_by'],
      dtype='object')

In [6]:
df['caller'].describe()

count          75707
unique          1931
top       Diane Chin
freq             971
Name: caller, dtype: object

In [7]:
df.dtypes

created               datetime64[ns]
caller                        object
opened_at             datetime64[ns]
opened_by                     object
priority                      object
state                         object
category                      object
subcategory                   object
assignment_group              object
bpl_location                  object
nyp_locaiton                  object
resolved                      object
assigned_to                   object
system                        object
closed_at                     object
closed_by                     object
mat_source                    object
reassignment_count             int64
reopen_count                   int64
resolved_by                   object
updated_on            datetime64[ns]
updated_by                    object
dtype: object

In [8]:
df['priority'].unique()

array(['3 - Moderate', '1 - Critical', '5 - Planning', '4 - Low', nan,
       '2 - High'], dtype=object)

In [9]:
df['state'].unique()

array(['Closed', 'Active', 'Awaiting User Info', 'Awaiting Vendor', 'New',
       'Resolved'], dtype=object)

In [10]:
df['category'].unique()

array(['Cataloging', 'Acquisitions', 'Selection', 'Logistics',
       'Offsite - ReCAP', 'Collections Processing', nan], dtype=object)

In [11]:
df['subcategory'].unique()

array(['Item attached to wrong bib record', 'Item not linked', 'Other',
       'Call number error', 'Cataloging error', 'Packing list error',
       'Materials not processed with labels, etc.',
       'Holds fulfillment--NYPL only', 'Item record problem',
       'Volume record needed', 'Duplicate records',
       'Supplies: Circ bins, discard bins, bin ties, tubs, etc.',
       'Call number missing', 'Location code error',
       'Shipment delivery error', 'Status of request',
       'Pickup or delivery request', 'Spine labels', 'Damaged material',
       'Invoice payment', 'Materials received without paperwork',
       'Author and submission inquiries',
       'Barcodes for circulating materials', 'Collection HQ',
       'Online Catalog', 'Gift books or Donations--NYPL only',
       'Barcoding', 'OCLC Holdings', 'Delivery issue or error',
       'New Item/Title Request', 'Electronic resource',
       'Damaged material / missing pieces', 'BiblioCommons',
       'Leased items', 'Book ja

In [12]:
df['reassignment_count'].describe()

count    75724.000000
mean         0.110916
std          0.364769
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          8.000000
Name: reassignment_count, dtype: float64

In [13]:
adf_out = pd.DataFrame(columns=['dept', 'year', 'tickets'])

In [14]:
years = [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
for agroup, adf in df.groupby('category'):
    d = dict()
    for y, ydf in adf.groupby(adf['created'].map(lambda x: x.year)):
        d[y]={'dept': agroup, 'year': y, 'tickets': ydf.shape[0]}
    for y in years:
        if y in d.keys():
            adf_out = adf_out.append(d[y], ignore_index=True)
        else:
            adf_out = adf_out.append({'dept': agroup, 'year': y, 'tickets': 0}, ignore_index=True)         

In [15]:
adf_out.head()

,dept,year,tickets
0,Acquisitions,2013,110
1,Acquisitions,2014,139
2,Acquisitions,2015,185
3,Acquisitions,2016,290
4,Acquisitions,2017,325


In [16]:
adf_out.to_csv('../data-display/tickets-per-dept-yearly.csv', index=False)

In [17]:
# open, awaiting tickets per department
odf = df[(df['state']!='Closed')&(df['state']!='Resolved')]

In [18]:
odf.shape

(3409, 22)

In [19]:
odf_out = pd.DataFrame(columns=['dept', 'year', 'tickets'])
years = [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
for agroup, adf in odf.groupby('category'):
    d = dict()
    for y, ydf in adf.groupby(adf['created'].map(lambda x: x.year)):
        d[y]={'dept': agroup, 'year': y, 'tickets': ydf.shape[0]}
    for y in years:
        if y in d.keys():
            odf_out = odf_out.append(d[y], ignore_index=True)
        else:
            odf_out = odf_out.append({'dept': agroup, 'year': y, 'tickets': 0}, ignore_index=True)

In [20]:
odf_out.head()

,dept,year,tickets
0,Acquisitions,2013,13
1,Acquisitions,2014,20
2,Acquisitions,2015,32
3,Acquisitions,2016,53
4,Acquisitions,2017,65


In [21]:
odf_out.to_csv('../data-display/unresolved-per-dept-yearly.csv', index=False)

In [78]:
# filter out tickets that did not end up in cataloging
cdf = df[df['assignment_group']=='BKOPS CAT']

In [79]:
cdf.shape

(14470, 22)

In [80]:
cdf['assignment_group'].unique()

array(['BKOPS CAT'], dtype=object)

In [81]:
cdf['category'].unique()
states = ['Closed', 'Resolved', 'Awaiting User Info', 'Awaiting Vendor', 'Active', 'New']

In [82]:
sdf_out = pd.DataFrame(columns=['year', 'state', 'tickets', 'stack_order'])
for state, sdf in cdf.groupby('state'):
    d = dict()
    for y, ydf in sdf.groupby(sdf['created'].map(lambda x: x.year)):
        d[y] = {'year': y, 'state': state, 'tickets': ydf.shape[0], 'stack_order': states.index(state)}
    for y in years:
        if y in d.keys():
            sdf_out = sdf_out.append(d[y], ignore_index=True)
        else:
            sdf_out = sdf_out.append({'year': y, 'state': state, 'tickets': 0, 'stack_order': states.index(state)}, ignore_index=True)

In [83]:
sdf_out.shape

(48, 4)

In [84]:
sdf_out.to_csv('../data-display/cat-tickets-per-state-yearly.csv', index=False)

In [85]:
# misassigned tickets in CAT
vcdf = df[(df['category'] == 'Cataloging')&(df['assignment_group'] != 'BKOPS CAT')|(df['assignment_group']=='BKOPS CAT')]

In [86]:
vcdf.shape[0]

15070

In [87]:
vcdf['valid'] = vcdf.apply(lambda row: 'correct' if row.assignment_group == 'BKOPS CAT' else 'misassigned', axis=1)

c:\users\tomaszkalata\envs\servicenowanalysisenv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [88]:
vcdf['valid'].value_counts()

correct        14470
misassigned      600
Name: valid, dtype: int64

In [89]:
vcdf_out = pd.DataFrame(columns=['year', 'valid', 'tickets'])

In [90]:
for valid, vdf in vcdf.groupby('valid'):
    d = dict()
    for y, ydf in vdf.groupby(vdf['created'].map(lambda x: x.year)):
        d[y] = {'year': y, 'valid': valid, 'tickets': ydf.shape[0]}
    for y in years:
        if y in d.keys():
            vcdf_out = vcdf_out.append(d[y], ignore_index=True)
        else:
            vcdf_out = vcdf_out.append({'year': y, 'valid': valid, 'tickets': 0}, ignore_index=True)

In [91]:
vcdf_out.head()

,year,valid,tickets
0,2013,correct,1201
1,2014,correct,2018
2,2015,correct,2678
3,2016,correct,2471
4,2017,correct,2144


In [92]:
vcdf_out.to_csv('../data-display/cat-tickets-misassigned.csv', index=False)

## Average open and resolved per month

In [101]:
rcdf = cdf[cdf['resolved'].notnull()]
rcdf.shape[0]

12004

In [99]:
total_tickets = cdf.shape[0]

In [100]:
total_tickets

14470

In [105]:
for year, ydf in cdf.groupby(cdf['created'].map(lambda x: x.year)):
    for month, mdf in ydf.groupby(ydf['created'].map(lambda m: m.month)):
        print(year, month, mdf.shape[0])
        print(mdf.shape[0] / 30)

2013 5 174
5.8
2013 6 132
4.4
2013 7 141
4.7
2013 8 187
6.233333333333333
2013 9 113
3.7666666666666666
2013 10 148
4.933333333333334
2013 11 163
5.433333333333334
2013 12 143
4.766666666666667
2014 1 166
5.533333333333333
2014 2 156
5.2
2014 3 162
5.4
2014 4 187
6.233333333333333
2014 5 140
4.666666666666667
2014 6 101
3.3666666666666667
2014 7 138
4.6
2014 8 307
10.233333333333333
2014 9 142
4.733333333333333
2014 10 190
6.333333333333333
2014 11 147
4.9
2014 12 182
6.066666666666666
2015 1 188
6.266666666666667
2015 2 173
5.766666666666667
2015 3 295
9.833333333333334
2015 4 284
9.466666666666667
2015 5 224
7.466666666666667
2015 6 245
8.166666666666666
2015 7 199
6.633333333333334
2015 8 245
8.166666666666666
2015 9 191
6.366666666666666
2015 10 224
7.466666666666667
2015 11 190
6.333333333333333
2015 12 220
7.333333333333333
2016 1 296
9.866666666666667
2016 2 200
6.666666666666667
2016 3 197
6.566666666666666
2016 4 201
6.7
2016 5 196
6.533333333333333
2016 6 182
6.06666666666666